In [ ]:
%load_ext autoreload


In [41]:
from random import random, randint
import numpy as np
from copy import deepcopy

In [8]:
a=2
b=3
2+3

5

In [1]:
etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]


In [3]:
len(etapa_dif)

31

In [109]:
import numpy as np
from random import random, randint, randrange, choices

NUM_LINES = 31
NUM_COLUMNS = 7
ENERGY = 8
MAX_POPULATION = 2000
etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]

def checa_vazio(m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1

def garante_vivo(m):
    if m.sum() < NUM_COLUMNS*ENERGY:
        return m
    # muda uma vida pois máximo possivel por construção é NUM_COLUMNS*ENERGY
    i = randint(0, NUM_LINES - 1)
    while m[i].sum() < 2:
        i = randint(0, NUM_LINES - 1)
    j = randint(0, NUM_COLUMNS - 1)
    while m[i][j] == 0:
        j = randint(0, NUM_COLUMNS - 1)
    m[i][j] = 0
    return m

def checa_vivo(m):
    if m.sum() < NUM_COLUMNS*ENERGY:
        return -1
    return 0

# talvez usar garante_vivo
def cria_novo_matrix():
    m=np.zeros((NUM_LINES, NUM_COLUMNS))
    vivo = randint(0, NUM_COLUMNS-1)
    for i in range(NUM_COLUMNS):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < ENERGY:
         n = randint(0, NUM_LINES - 1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(m)
    while index != -1:
        n = randint(0, NUM_COLUMNS - 1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, NUM_LINES - 1)
        while m[k][n]== 0:
            k = randint(0, NUM_LINES - 1)
        m[k][n] = 0
        index = checa_vazio(m)
    return m 

def custo_tempo(matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix) # returna tempo total de todas etapas

def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = list()
    for person in population:
        times.append(person[1])
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] # (person, time)

def random_selection_pair(population):
    times = [float(population[i][1]) for i in range(len(population))]
    times = np.divide(1, times) 
    return choices(population, weights=times, k=2)

def best_individual(population):
    print(population[0][0])
    population.sort(key=lambda y: y[1]) #sort by time
    best_time = population[0][1]
    best_person = population[0][0]
    return (best_time, best_person)

def cria_prob(lista):
    probs = np.divide(lista, np.sum(lista)) ## normaliza 
    lista_probs = []
    psum = 0
    for i in range(len(probs)):
        psum += probs[i]
        lista_probs.append(psum)
    return lista_probs

def die(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1]) ## ordena
    times = [float(population[i][1]) for i in range(len(population))]
    for i in range(1,excess+1):
        lista_probs = cria_prob(times)
        p = random()
        j = busca(p, lista_probs)
        while j < 10: # nao mata 10 primeiros
            p = random()
            j = busca(p, lista_probs)
        population.pop(j)
        times.pop(j)
    # return population

def die_optimal(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1]) ## ordena
    times = [float(population[i][1]) for i in range(len(population))]
    deaths = choices(population[10:], weights = times[10:], k = excess)
    for i in range(1,excess+1):
        lista_probs = cria_prob(times)
        p = random()
        j = busca(p, lista_probs)
        while j < 10: # nao mata 10 primeiros
            p = random()
            j = busca(p, lista_probs)
        population.pop(j)
        times.pop(j)

def mutation(person, probability = 0.3):
    if random() < probability:
        kshift = randrange(NUM_LINES)
        person = np.roll(person, NUM_COLUMNS*kshift)
    return person

def swap(person):
    new_person = deepcopy(person)
    p1 = randrange(NUM_COLUMNS)
    p2 = p1
    while p2 == p1:
        p2 = randrange(NUM_COLUMNS)

    for s1 in range(NUM_LINES):
        for s2 in range(s1, NUM_LINES):
            swaped = deepcopy(new_person[0])
            swaped[s1][p1] = new_person[0][s1][p2]
            swaped[s1][p2] = new_person[0][s1][p1]
            swaped[s2][p2] = new_person[0][s2][p1]
            swaped[s2][p1] = new_person[0][s2][p2]
            if checa_vazio(swaped) == -1 and checa_vivo(swaped)==-1:
                swaped = garante_vivo(swaped)
                tempo_swaped = custo_tempo(swaped)
                if tempo_swaped < new_person[1]:
                    new_person = (swaped, tempo_swaped)
    return new_person
     
def repetition(population, potential_child):
    for child in population:
        if np.array_equal(child[0], potential_child[0]):
            return 1
    return 0

# mae  = matrix
def reproduce(mae, pai):
    childs = []
    child1 = np.zeros((NUM_LINES, NUM_COLUMNS))
    child2 = np.zeros((NUM_LINES, NUM_COLUMNS))
    for i in range (NUM_COLUMNS):
        if randint(0,1):
           child1[:, i] = mae[:, i]
           child2[:, i] = pai[:, i]
        else:
            child1[:, i] = pai[:, i]
            child2[:, i] = mae[:, i]
    if checa_vazio(child1) == -1:
        child1 = garante_vivo(child1)
        childs.append(child1)
        if checa_vazio(child2) == -1:
            child2 = garante_vivo(child2)
            childs.append(child2)
    elif checa_vazio(child2) == -1:
        child2 = garante_vivo(child2)
        childs.append(child2)
    else:
        return reproduce(mae,pai)
    return childs


def genetic_algorithm(population): ## population: (persosn, time)
    count_stop = 0
    iter = 0
    while count_stop < 1:
        population.sort(key=lambda person: person[1]) #sort by time
        best_time = population[0][1]
        best_individual = population[0][0]

        
        print('iter', iter)
        print('count_stop', count_stop)
        new_population = population[0:500] #mantém melhores 500 habitantes ## chegou a 1923 em 4 min
        #new_population = population
        for i in range(len(population)//2):
            parents = random_selection_pair(population) # [(person1, time1), (person2, time2)]
            
            childs = reproduce(parents[0][0], parents[1][0])
            for child in childs:
                child = mutation(child)        
                if repetition(new_population, child) == 0:
                    tempo_child = custo_tempo(child)
                    new_population.append((child, tempo_child))


        population = new_population
        print('new pop', len(population))
        if len(population) > MAX_POPULATION:
            die(population, MAX_POPULATION)
            print('new pop after massive deaths', len(population))
        
        population.sort(key=lambda person: person[1])
        for i, genome in enumerate(population[0:2]): # Realiza o swap nas duas melhores soluções
            swaped = swap(genome) 
            population[i] = swaped
        population.sort(key=lambda person: person[1])
        time = population[0][1]
        individual = population[0][1]
        if time < best_time:
            best_time = time
            best_individual = individual
            count_stop = 0
        
        else:
            count_stop += 1
        print('time', time)
        print('best_time', best_time)    
    return best_individual

def initial_population(n):
    population = list()
    count = 0
    while count < n:
        potential_person = cria_novo_matrix()
        if repetition(population, potential_person) == 0:
            tempo = custo_tempo(potential_person)
            population.append((potential_person, tempo))
            count +=1
    return population

In [122]:
import numpy as np
from random import random, randint

NUM_LINES = 5
NUM_COLUMNS = 4
ENERGY = 2

etapa_dif = [10,20,50, 5,  30]
personagem_agilidade = [1,2,3,4]
potential_person = cria_novo_matrix()
tempo = custo_tempo(potential_person)
person = (potential_person, tempo)

In [110]:
population = initial_population(500)

In [124]:
person[0]

array([[0., 1., 1., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [126]:
person

(array([[0., 1., 1., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 77.77777777777777)

In [125]:
a = swap(person)
a

(array([[0., 1., 1., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]]),
 50.27777777777778)

In [128]:
a[0].sum()

7.0

In [129]:
person[0].sum()

7.0

In [119]:
NUM_LINES

5

In [111]:
ind = genetic_algorithm(population)

iter 0
count_stop 0
new pop 834
time 2131.901262526902
best_time 2131.901262526902
iter 0
count_stop 0
new pop 1033
time 2085.8894454396204
best_time 2085.8894454396204
iter 0
count_stop 0
new pop 1152
time 2083.37127812708
best_time 2083.37127812708
iter 0
count_stop 0
new pop 1246
time 1694.7279931461296
best_time 1694.7279931461296
iter 0
count_stop 0
new pop 1274
time 1694.7279931461296
best_time 1694.7279931461296


In [116]:
swap(potential_person)

IndexError: invalid index to scalar variable.

In [112]:
ind.sum()

67.0

In [64]:
randint(1,10)

3

In [6]:
nlines = 31
ncols = 7
k = 8

def cria_linha(ncols): #ncols = 7
    l=[]
    i=0
    t=0
    while i<ncols:
        n=randint(0,1) # mudar probabilidade!!!!!!!! assim nao funciona
        l.append(n)
        t+=n
        i+=1
    if t:#checa se pelo menos um membro percorreu o percurso
        return l
    else:
        return cria_linha(ncols)
   
#cria matriz [30][7]
def cria_matriz(nlines, ncols): # nlines=30, ncols = 8
    i=0
    m=[]
    while i<nlines:
       m.append(cria_linha(ncols))
       i+=1
    return m


def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    vivo = randint(0, ncols-1)
    for i in range(ncols):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < k:
         n = randint(0,nlines-1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0,ncols-1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, nlines-1)
        while m[k][n]== 0:
            k = randint(0, nlines-1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 


In [ ]:
nlines = 31
ncols = 7
k = 8
#checa se alguem foi usado mais que k vezes
#checa se alguem foi usado mais que k vezes
def checa_k_vivo(*m):
    alive = 0
    for i in range(len(m[0])):
        t=0
        for j in range(len(m)):
            t+=(m[j][i])
            # j+=1z
        alive += t
        if t>k:
            return i #retorna indice do que apareceu mais de k vezes
    if alive >= k*len(m[0]):
        return -2
            
        # i+=1
    return -1


def checa_vazio(m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1
    
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    # x = (person, time)
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[0][line][col] != y[0][line][col]:
                tuplas.append((line, col))
    casas_sorteadas = []
    n = min(5, len(tuplas))
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(num)
    childs = [x, y]
    bits = np.zeros(n)
    bits[0] = 1
    m = deepcopy(x[0])
    l = pow(2,n)
    for i in range(l-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[0][a][b]
        um = np.zeros(n)
        um[0] = 1
        bits = soma_bits(bits, um, n)
        if checa_k_vivo(*matrix) == -1 and checa_vazio(*matrix) == -1:
            time = custo_tempo(etapa_dif, personagem_agilidade, matrix)
            childs.append((matrix, time))
    return childs

In [ ]:

etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]
nlines = 31
ncols = 7
k = 8
# custo do tempo a cada etapa: dificuldade/ agilidade na etapa
def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix)



def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = list()
    for person in population:
        times.append(person[1])
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] # (person, time)
    
def best_individual(population):
    population.sort(key=lambda y: y[1]) #sort by time
    best_time = population[0][1]
    best_person = population[0][0]
    return best_time, best_person

def die(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1])
    for i in range(1,excess+1):
        population.pop(tam-i)
    # return population

def repetition(population, potential_child):
    for child in population:
        if np.array_equal(child[0], potential_child[0]):
            return 1
    return 0

def genetic_algorithm(population): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)
    count = 0
    while count < 50:
    ## tamanho pop
    # ideia tirar caras ruins
        new_population = []
        # evaluate best individual...
        for i in range(200):
            x = random_selection(population) # (person, time)
            y = random_selection(population)
            while np.array_equal(x[0],y[0]):
                y = random_selection(population)
            childs = reproduce1(x,y)
            for child in childs:
                if repetition(new_population, child) == 0:
                    new_population.append(child)
            # p = random()
            # if (p < 0.01):
            #     child = mutate(child)
        population = new_population
        print('new pop', len(population))
        if len(population) > 3000:
            die(population, 3000)
            print('new pop after massive deaths', len(population))
        
        time, ind = best_individual(population)
        if time < best_time:
            best_time = time
            fit_ind = ind
            print('time', time)
            print('best_time', best_time)
        else:
            count += 1
            
    return fit_ind

In [ ]:
ENERGY = 2

In [ ]:
def reproduce(mae, pai):
    m = np.zeros((NUM_LINES, NUM_COLUMNS))
    for i in range (NUM_COLUMNS):
        if randint(0,1):
           m[:, i]=mae[:, i]
        else:
            m[:, i]=pai[:, i]
    if checa_vazio(m) != -1: 
        return reproduce(mae,pai)
    m = garante_vivo(m)
    return m

In [ ]:
def garante_vivo(m):
    if m.sum() < NUM_COLUMNS*ENERGY:
        return m
    # muda uma vida pois máximo possivel por construção é NUM_COLUMNS*ENERGY
    i = randint(0, NUM_LINES - 1)
    while m[i].sum() < 2:
        i = randint(0, NUM_LINES - 1)
    j = randint(0, NUM_COLUMNS - 1)
    while m[i][j] == 0:
        j = randint(0, NUM_COLUMNS - 1)
    m[i][j] = 0
    return m

In [ ]:
population = list()
count = 0
while count < 100:
    potential_person = cria_novo_matrix()
    if repetition(population, potential_person) == 0:
        tempo = custo_tempo(etapa_dif, personagem_agilidade, potential_person)
        population.append((potential_person, tempo))
        count +=1

In [ ]:
len(population)

In [ ]:

def cria_novo_matrix(k):
    m=np.zeros((NUM_LINES, NUM_COLUMNS))
    vivo = randint(0, NUM_COLUMNS-1)
    for i in range(NUM_COLUMNS):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < k:
         n = randint(0, NUM_LINES - 1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0, NUM_COLUMNS - 1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, NUM_LINES - 1)
        while m[k][n]== 0:
            k = randint(0, NUM_LINES - 1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 

In [47]:
NUM_LINES = 4
NUM_COLUMNS = 3
ENERGY = 2
M = cria_novo_matrix()

In [48]:
M

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 1., 0.]])

In [51]:
np.roll(M,3)

array([[1., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
lista = [1,2]
lista2 = [3,2]
lista2.append(lista)
lista2

In [50]:
M

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 1., 0.]])

In [ ]:
genetic_algorithm(population)

In [ ]:
time, ind = best_individual(population)

In [ ]:
time

In [ ]:
min(4,1)

In [ ]:
x = population[0]
y= population[1]

In [ ]:
x

In [ ]:
y

In [ ]:
childs= reproduce1(x,y,2)

In [ ]:
len(childs)

In [ ]:
np.array_equal(childs[1], childs[2])

In [ ]:
a = genetic_algorithm(population, 40)

In [ ]:
from random import random

In [ ]:
m = np.matrix([[0,1], [1,1], [0,0]])
checa_vazio(*m)

In [ ]:
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits


In [ ]:
bits(16)

In [ ]:
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r